In [38]:
import requests
import pytz

import json
from datetime import datetime, time as datetime_time, timedelta



In [39]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r


In [53]:
def make_api_request(stop_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/OperationMonitoring/4.0/PassingTimeByPoint/" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [54]:
def parse_waiting_times(json_payload, destination):
    passing_times = json_payload['points'][0]['passingTimes']
    str_expected_arrival_times = [passing_time['expectedArrivalTime'] for passing_time in passing_times if passing_time['destination']['fr']==destination]
    print(str_expected_arrival_times)
    expected_arrival_times = [datetime.fromisoformat(str_expected_arrival_time) for str_expected_arrival_time in str_expected_arrival_times]
    return expected_arrival_times 

In [55]:
def compute_time_diff(start, end):
    if isinstance(start, datetime_time): # convert to datetime
        assert isinstance(end, datetime_time)
        start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
    if start <= end: # e.g., 10:33:26-11:15:49
        return end - start
    else: # end < start e.g., 23:55:00-00:25:00
        end += timedelta(1) # +day
        assert end > start
        return end - start

In [57]:
legrand_stop_id = "1047"
bearer = '92933b65187eb1ac049b9132c1afa7cc'
response = make_api_request(legrand_stop_id, bearer)


In [58]:
expected_arrival_times = parse_waiting_times(response, 'ROODEBEEK' )

current_time_utc = datetime.utcnow()
current_time_be = pytz.utc.localize(current_time_utc)
tram_arrival_time = expected_arrival_times[0]

waiting_time = compute_time_diff(current_time_be, tram_arrival_time)
print(waiting_time)

['2020-03-23T15:24:00+01:00']
0:33:47.819793
